In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn import cross_validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score
import xgboost

C:\Users\Dell\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
os.chdir(r"C:\Users\Dell\Desktop\Finalproject")

In [3]:
train = pd.read_csv("train_loan.csv")

In [4]:
test = pd.read_csv("test_loan.csv")

In [5]:
#splitting
y_train = np.array(train['default_ind'])  
y_test = np.array(test['default_ind'])
X_train = train.drop('default_ind', axis = 1)  
X_train = np.array(X_train)
X_test = test.drop('default_ind', axis = 1)
X_test = np.array(X_test)

In [6]:
# normalizing data
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)
X_train=scaler.transform(X_train)

In [7]:
scaler=StandardScaler()
scaler.fit(X_test)
X_test=scaler.transform(X_test)

In [8]:
scaler=StandardScaler()
scaler.fit(X_test)
X_test=scaler.transform(X_test)

In [9]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [10]:
#up_sampling 
from sklearn.utils import resample

In [12]:
# Separate majority and minority classes for train
df_majority_train = train[train.default_ind==0]
df_minority_train = train[train.default_ind==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority_train, 
                                 replace=True,     # sample with replacement
                                 n_samples=199733,    # to match majority class
                                 random_state=111) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled_train = pd.concat([df_majority_train, df_minority_upsampled])
 
# Display new class counts
df_upsampled_train.default_ind.value_counts()

1    199733
0    199733
Name: default_ind, dtype: int64

In [13]:
from sklearn.linear_model import LogisticRegression
# Separate input features (X) and target variable (y)
y_train1 = df_upsampled_train.default_ind
X_train1 = df_upsampled_train.drop('default_ind', axis=1)
y_train1 = np.array(y_train1)
X_train1 = np.array(X_train1)

In [14]:
classifier=(LogisticRegression())
#fitting traing data to model
classifier.fit(X_train1,y_train1)
Y_pred=classifier.predict(X_test)

In [15]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
cfm=confusion_matrix(y_test,Y_pred)
print(cfm)
print("Classification report :")
print(classification_report(y_test,Y_pred))
accuracy_score=accuracy_score(y_test,Y_pred)
print("Accuracy of the model:",accuracy_score)
#auc curve 
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, Y_pred)
auc(false_positive_rate, true_positive_rate)

[[4516 3532]
 [  87  162]]
Classification report :
             precision    recall  f1-score   support

          0       0.98      0.56      0.71      8048
          1       0.04      0.65      0.08       249

avg / total       0.95      0.56      0.69      8297

Accuracy of the model: 0.563818247559


0.60586780521689132

In [16]:
#%% Adjusting threshold
#store the predicted probabilities
y_pred_prob = classifier.predict_proba(X_test)
print(y_pred_prob)

[[ 0.50575012  0.49424988]
 [ 0.51704498  0.48295502]
 [ 0.50030051  0.49969949]
 ..., 
 [ 0.48140435  0.51859565]
 [ 0.49338639  0.50661361]
 [ 0.51996448  0.48003552]]


In [31]:
y_pred_class=[]
for value in y_pred_prob[:,0]:
    if value < 0.495:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0)

In [32]:
from sklearn.metrics import confusion_matrix,accuracy_score
cfm=confusion_matrix(y_test.tolist(),y_pred_class)
print(cfm)
accuracy_score=accuracy_score(y_test.tolist(),y_pred_class)
print("Accuracy of the model: ",accuracy_score)
print("Classification report :")
print(classification_report(y_test,y_pred_class))
#auc curve 
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred_class)
auc(false_positive_rate, true_positive_rate)

[[5234 2814]
 [ 104  145]]
Accuracy of the model:  0.648306616849
Classification report :
             precision    recall  f1-score   support

          0       0.98      0.65      0.78      8048
          1       0.05      0.58      0.09       249

avg / total       0.95      0.65      0.76      8297



0.61633861489696362

In [33]:
#decision tree
from sklearn.tree import DecisionTreeClassifier

model_DecisionTree=DecisionTreeClassifier()
model_DecisionTree.fit(X_train1,y_train1)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [34]:
Y_pred=model_DecisionTree.predict(X_test)

In [35]:
from sklearn.metrics import confusion_matrix,accuracy_score
cfm=confusion_matrix(y_test.tolist(),Y_pred)
print(cfm)
accuracy_score=accuracy_score(y_test.tolist(),Y_pred)
print("Accuracy of the model: ",accuracy_score)
print("Classification report :")
print(classification_report(y_test,Y_pred))
#auc curve 
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, Y_pred)
auc(false_positive_rate, true_positive_rate)

[[8048    0]
 [ 249    0]]
Accuracy of the model:  0.969989152706
Classification report :
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      8048
          1       0.00      0.00      0.00       249

avg / total       0.94      0.97      0.96      8297



C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.5

In [36]:
#using xgboost
import xgboost

In [37]:
from xgboost import XGBRegressor
my_model = XGBRegressor(learning_rate =0.09,subsample=0.5,colsample_bytree=0.85,reg_alpha=0.005,min_child_weight = 5)

In [38]:
my_model.fit(X_train1, y_train1, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.85, gamma=0, learning_rate=0.09,
       max_delta_step=0, max_depth=3, min_child_weight=5, missing=None,
       n_estimators=100, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0.005, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.5)

In [39]:
test_y_pred = my_model.predict(X_test)
auc = roc_auc_score(y_test, test_y_pred)
print("Performance  test : ", auc)

Performance  test :  0.568172790566
